# Housing Data Enrichment Demo

This notebook demonstrates how to use API services to add new attributes to housing listing data.


In [ ]:
import pandas as pd
import asyncio
from services.census import median_income_from_latlon
from services.mtd import bus_stops_within_1km
from services.police_beats import police_beat_for_point
from services.google_places import count_places_nearby
from services.google_routes import drive_time_minutes


## 1. Load Data and Test Single Listing

Load the data and test the first listing to verify that all APIs are working correctly.


In [ ]:
# Load housing listings data
df = pd.read_csv('../data/listings_details_allcities.csv')
print(f"Loaded {len(df)} listings")

# Get the first listing for testing
first_listing = df.iloc[0]
lat = first_listing['latitude']
lon = first_listing['longitude']
address = first_listing['address']

print(f"\nTest listing: {address}")
print(f"Coordinates: ({lat}, {lon})")


In [ ]:
# Test all APIs
print("Testing Census API...")
median_income = await median_income_from_latlon(lat, lon)
print(f"✓ Median household income: ${median_income:,}" if median_income else "- No data")

print("\nTesting MTD API...")
bus_stops = await bus_stops_within_1km(lat, lon)
print(f"✓ Bus stops within 1km: {bus_stops}")

print("\nTesting Police Beats API...")
police_beat = await police_beat_for_point(lat, lon)
print(f"✓ Police beat: {police_beat}" if police_beat else "- Not within any police beat")

print("\nTesting Google Places API...")
restaurants = await count_places_nearby(lat, lon, included_types=["restaurant"], radius_m=1000)
print(f"✓ Nearby restaurants: {restaurants}")

print("\nTesting Google Routes API...")
uiuc_coords = (40.110244, -88.227258)
drive_time = await drive_time_minutes((lat, lon), uiuc_coords, mode="DRIVE")
print(f"✓ Drive time to UIUC Main Quad: {drive_time:.1f} minutes" if drive_time else "- No route data")


## 📝 Next Steps

If the API tests are successful, you can process the complete dataset using the command line script:

```bash
# Process all listings with default settings
python code/enrich_listings.py -i data/listings_details_allcities.csv -o data/listings_enriched.csv

# Skip expensive Google APIs (use only free APIs)
python code/enrich_listings.py -i data/listings.csv -o data/enriched.csv --skip-places --skip-routes

# Custom batch size and search radius
python code/enrich_listings.py -i data/listings.csv -o data/enriched.csv --batch-size 10 --places-radius 500

# Resume from a specific index if interrupted
python code/enrich_listings.py -i data/listings.csv -o data/enriched.csv --start-idx 100
```

For more information, see:
- `code/ENRICHMENT_README.md` - Detailed documentation
